In [1]:
import pandas as pd

import torch
import torch.nn as nn
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from helper_functions import normalize, accuracy_fnn
from torch.utils.data import DataLoader, TensorDataset 
from CNN import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device =', device)

c:\Program Files\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


device = cpu


In [2]:
from DataLoader import Loaders



df = Loaders(batch_size = None ,normalization= True,file_path='future_clean/future-5/', dimension=5)

In [3]:
pd.read_parquet(r'future_clean\future-5\train.parquet')

,unit1_pwr_60ghz_1,unit1_pwr_60ghz_2,unit1_pwr_60ghz_3,unit1_pwr_60ghz_4,unit1_pwr_60ghz_5,unit1_pwr_60ghz_6,unit1_pwr_60ghz_7,unit1_pwr_60ghz_8,blockage_1,blockage_2,blockage_3,blockage_4,blockage_5
0,0.000098,0.000094,0.000095,0.000087,0.000094,0.000101,0.000131,0.000048,1,1,1,0,0
1,0.000471,0.000417,0.000445,0.000295,0.000442,0.000426,0.000529,0.000098,1,1,1,0,0
2,0.000421,0.000447,0.000420,0.000389,0.000408,0.000413,0.000579,0.000131,1,1,1,0,0
3,0.000409,0.000386,0.000395,0.000399,0.000388,0.000377,0.000572,0.000145,1,1,1,0,0
4,0.000249,0.000328,0.000242,0.000263,0.000263,0.000228,0.000293,0.000128,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100219,0.000189,0.000168,0.000247,0.000216,0.000181,0.000217,0.000190,0.000201,0,0,0,0,0
100220,0.000316,0.000299,0.000336,0.000337,0.000295,0.000338,0.000325,0.000321,0,0,0,0,0
100221,0.000319,0.000253,0.000260,0.000242,0.000241,0.000191,0.000265,0.000295,0,0,0,0,0
100222,0.000111,0.000098,0.000091,0.000083,0.000085,0.000076,0.000092,0.000087,0,0,0,0,0


In [4]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [5]:
def training_loop( model: torch.nn.Module,
                      data_loader_train: torch.utils.data.DataLoader,
                      data_loader_test: torch.utils.data.DataLoader,
                      loss_fn: torch.nn.Module,
                      optimizer: torch.optim.Optimizer,
                      accuracy_fn,
                      device: torch.device,
                      epochs: int
                      ,inplace=False):
        """Treina e testa um modelo PyTorch.

     Passa um modelo PyTorch de destino por meio de train_step() e test_step()
     funções para um número de épocas, treinando e testando o modelo
     no mesmo loop de época.

     Calcula, imprime e armazena métricas de avaliação.

     Argumentos:
       model: um modelo PyTorch a ser treinado e testado.
       train_dataloader: Uma instância do DataLoader para o modelo a ser treinado.
       test_dataloader: Uma instância do DataLoader para o modelo a ser testado.
       otimizador: Um otimizador PyTorch para ajudar a minimizar a função de perda.
       loss_fn: uma função de perda do PyTorch para calcular a perda em ambos os conjuntos de dados.
       epochs: Um número inteiro indicando para quantas épocas treinar.
       dispositivo: um dispositivo de destino para calcular (por exemplo, "cuda" ou "cpu").
      
     Retorna:
       Um dicionário de perda de treinamento e teste, bem como treinamento e
       testar métricas de precisão. Cada métrica tem um valor em uma lista para
       cada época.
       Na forma: {train_loss: [...],
                 train_acc: [...],
                 teste_perda: [...],
                 test_acc: [...]}
       Por exemplo, se o treinamento for epochs=2:
               {train_loss: [2.0616, 1.0537],
                 train_acc: [0,3945, 0,3945],
                 perda_teste: [1.2641, 1.5706],
                 test_acc: [0,3400, 0,2973]}
     """

        print(f"Training on {device}")
        for epoch in tqdm(range(epochs)):

            
            print(f" Epoch: {epoch}\n---------")
            # loop pelo dataloader de treino
            model.to(device)
            model.train().double()
            training_loss = 0
            training_accurary = 0
            valid_loss = 0
            for batch, (inputs, target) in enumerate(data_loader_train):
                # movendo os dados para o dispositivo de processamento
                inputs = inputs.to(device).double()
                inputs = inputs.unsqueeze(1)
                target = target.unsqueeze(1)
                
                # fazendo as previsões
                output = model(inputs.double())

                # calculando a perda
                loss = loss_fn(output, target)

                training_loss += loss.data.item()
                training_accurary += accuracy_fn(target, output.argmax(dim=1).unsqueeze(1))

                # retropropagando os gradientes e atualizando os pesos
                optimizer.zero_grad()

                loss.backward()

                optimizer.step()

            training_loss /= len(data_loader_train)
            training_accurary /= len(data_loader_train)

            # imprimindo as métricas de treino a cada 50 lotes
            
            print(
                    f'Train loss: {training_loss:.5f} | Train accuracy: {training_accurary:.2f}%')

            # avaliando o modelo no dataloard de teste
            # loop pelo dataloader de teste

            model.eval().double()
            valid_loss = 0
            test_loss = 0
            test_accurary = 0
            with torch.inference_mode():

                for data, target in data_loader_test:
                    # movendo os dados para o dispositivo de processamento
                    data = data.to(device)
                    target = target.to(device)
                    target = target.unsqueeze(1)
                    
                    data = data.unsqueeze(1)
                    # 1. Forward pass
                    test_pred = model(data)
                    # 2. Calculate loss and accuracy
                    test_loss += loss_fn(test_pred, target)
                    valid_loss += test_loss.data.item()
                    test_accurary += accuracy_fn(target,
                                                 test_pred.argmax(dim=1).unsqueeze(1))

                valid_loss /= len(data_loader_test)
                test_accurary /= len(data_loader_test)

            
            
            print(
                    f'Test loss: {valid_loss:.5f} | Test accuracy: {test_accurary:.2f}%')


In [6]:
model_ = CNNModel(in_channels=8,out_channels=5)
loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.Adam(params=model_.parameters(), lr= 1e-03)

In [7]:
training_loop(model = model_,
                      data_loader_train= df.train_loader,
                      data_loader_test=df.test_loader,
                      loss_fn = loss_fn,
                      optimizer =optimizer,
                      accuracy_fn = accuracy_fnn,
                      device = device,
                      epochs =30
                      ,inplace=False)

Training on cpu


  0%|          | 0/30 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 0.00000 | Train accuracy: 71.95%
Test loss: 0.00000 | Test accuracy: 71.62%
 Epoch: 1
---------
Train loss: 0.00000 | Train accuracy: 71.95%
Test loss: 0.00000 | Test accuracy: 71.62%
 Epoch: 2
---------
